# Introduction

Citibike is the leading bikesharing platform in New York, including the districts of Manhattan, Brooklyn, Queens & Jersey City. That said, the company is far from being profitable due to growing competition from other modes of transportation (e.g., scooters, electric bikes, etc.). According to a consulting firm hired a few months ago, the operations of Citibike are inefficient due to the lack of data-driven decision processes. If Citibike were only 4\% more efficient, it would become profitable. During peak hours (e.g., morning commute time), the bikesharing platform is particularly unreliable. Approximately 20\% of the dock stations face a "stock-out" situation, meaning that there are no bikes available. In such cases, commuters choose other modes of transportation. 

<img src="citi-bike.jpg" width="500">

### Your assignment
The consulting firm recommends using **optimization models** to improve the operations, especially during peak time. As part of this new strategy, you are recently hired as a lead data scientist of Citibike. You suspect that the inventory of bikes and the dock stations are not well positioned given the incoming demand at peak times. In this workshop, you will develop a data-driven optimization model to improve the positioning of inventory at peak time. In particular, you will formulate and implement integer programming models.

*Workshop instructions*: **Please carefully read all the instructions below:**
- The code cells are partially filled. You need to complete and execute the code in each cell. *You can consult any material posted on Canvas to help you get started with the syntax. You can ask questions to the TAs and discuss with your teammates.*
- Make sure to answer all the questions along the way in a detailed manner.
- This is an individual assignment; you should differentiate your coding style, answers, commenting, etc. from those of your teammates.
- Only Q1-Q3.6 are meant to be covered in class. You are free to start working on the other questions, but you should not exchange with your teammates on all remaining questions.

*Submission*: Your submission should include the following:
- *Notebook:* An html file along with a .ipynb file **(85 pt)**
- *Executive summary:* A 1 pager report summarizing all your findings (at least 11 pt font, 1 inch margin, pdf/doc format). This report should take the form of an executive summary that combines elements of your analysis and business recommendations. Supporting evidence can be provided in an appendix, or by referencing the questions of the workshop. The report will be evaluated along 3 dimensions: clarity, scientific validity, practical relevance. **(15 pt)**


# Q1. Visualize the data (15 pt)

## Load the data

Your colleague has generated data sets that contains all the needed information about the demand and available inventory of bikes during *one hour at peak time*. Your initial analysis will be based on the following data sets:

- `demand.csv`: describes the number of bikes rented at each dock station (during one peak hour),
- `starting_inventory.csv`: describes the initial inventory of bikes at each dock station (before peak hour),
- `distances.csv`: describes the distance between any two dock stations (in km unit).


Ideally, the inventory of bikes available the beginning of the peak hour should approximately match the expected amount of demand. For example, if you expect 5 user requests in station A, and 5 user requests in station B in the next hour, the number of bikes in station A and B should be roughly equal, otherwise there is an imbalance of inventory. Using spatial visualization, you will check if the initial inventory is balanced or imbalanced.

*Execute the cells below. Read carefully all the comments*

In [119]:
# Import various packages
import pandas as pd
import numpy as np
import folium # visualisation package for spatial data (plot of maps)
import seaborn as sns # general visualization package 
import matplotlib.pyplot as plt # general visualization package 
#next command allows you to display the figures in the notebook
%matplotlib inline     

In [120]:
## In case folium is not installed, run the command below and import folium again
# ! conda install -c conda-forge folium --yes

Next, you need to load the data sets as dataframes using the Pandas function `pd.read_csv()`.

In [121]:
starting_inventory = pd.read_csv("starting_inventory.csv", index_col=0) 
# "index_col=0" is used so that the dataframe is indexed by the first column, which is the dock station ID
demand = pd.read_csv("demand.csv", index_col=0)
distances = pd.read_csv("distances.csv", index_col=0)
distances.columns = list(map(lambda x: int(x),distances.columns.tolist())) # the column names (dock IDs) are converted into integers
replenishment = pd.read_csv("replenishment.csv", index_col=0)


Using the command `.describe()`, you can familiarize yourself with the format of the dataframes, and  check if there are missing entries.

In [122]:
starting_inventory.describe()

# Examine the data
print("Starting Inventory Statistics:")
print(starting_inventory.describe())
print("\nDemand Statistics:")
print(demand.describe())
print("\nDistances Statistics:")
print(distances.describe())

Starting Inventory Statistics:
            count    latitude   longitude
count  689.000000  689.000000  689.000000
mean    30.724238   40.732527  -73.967780
std     70.308432    0.041163    0.024024
min     10.000000   40.655400  -74.025353
25%     10.000000   40.695807  -73.987030
50%     10.000000   40.728419  -73.968044
75%     19.000000   40.767100  -73.949450
max    739.000000   40.814394  -73.910651

Demand Statistics:
         latitude   longitude       count
count  689.000000  689.000000  689.000000
mean    40.732527  -73.967780   27.648766
std      0.041163    0.024024   56.377839
min     40.655400  -74.025353    1.000000
25%     40.695807  -73.987030    5.000000
50%     40.728419  -73.968044   10.000000
75%     40.767100  -73.949450   26.000000
max     40.814394  -73.910651  428.000000

Distances Statistics:
              119         120         127         143         144         146  \
count  689.000000  689.000000  689.000000  689.000000  689.000000  689.000000   
mean    

### Q1.0. What is the distance between the dock stations "120" and "146"? What is the maximum inventory of bikes initially available over all dock stations?

In [123]:
# Distance between stations 120 and 146
distance_120_146 = distances.loc[120, 146]
# Maximum initial inventory
max_inventory = starting_inventory['count'].max()

print(f"Distance between stations 120 and 146: {distance_120_146} km")
print(f"Maximum initial inventory: {max_inventory} bikes")

Distance between stations 120 and 146: 5.7893437175538125 km
Maximum initial inventory: 739 bikes


## Plot the heatmaps of demand and inventory

In the next cells, we will generate a heatmap of New York, representing the level of demand in different regions.

*The code is provided. Feel free to experiment with different parameters to better understand their respective roles.*

In [124]:
def generateBaseMap(default_location=[40.71, -73.96], default_zoom_start=12.5):
    '''
    This function generates a base map using the folium package
    
    Arguments:
    default_location -> latitude, longitude (a list)
    default_zoom_start -> level of zoom of the map (a positive number)
    '''
    base_map = folium.Map(location=default_location,control_scale=True, zoom_start=default_zoom_start)
    # 'Stamen Toner' is a type of map; by default the map is "OpenStreetMap"
    return base_map

The next cell will plot a heatmap of the demand.

In [125]:
# Import the HeatMap object constructor from the folium package
from folium.plugins import HeatMap 

# To begin, generate a base map, which will be edited next
base_map = generateBaseMap()

# The demand is converted into a list of t-uples, each describing one request point: (longitude, latitude, count)
# Copy the demand dataframe 
demand_agg = demand.copy()
# Rounding latitudes and longitudes to nearest 0.001 decimal
demand_agg.loc[:,['latitude', 'longitude']] = demand_agg.loc[:,['latitude', 'longitude']].round(3)
# Aggregating by latitude and longitude 
demand_agg= demand_agg[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum()
# Converting demand into a list
demand_list = demand_agg.reset_index().values.tolist()

# Add the desired heatmap to the base map
HeatMap(data= demand_list,radius=0, max_zoom=15,minOpacity=0).add_to(base_map)
base_map

Ideally, you would like to compare the expected demand to the initial inventory at the dock stations. To this end, you need to visualize the inventory of bikes initially available.

*By re-using the code above, construct a heatmap for the inventory of bikes in the city (`starting_inventory`). You need to edit the lines of code relative to the construction of the demand list.*

In [126]:
# Uncomment and fill the code below

from folium.plugins import HeatMap
base_map = generateBaseMap()
inventory_agg = starting_inventory.copy()
# Round latitudes and longitudes
inventory_agg.loc[:,['latitude', 'longitude']] = inventory_agg.loc[:,['latitude', 'longitude']].round(3)
# Aggregate inventory by location
inventory_agg = inventory_agg[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum()
# Convert to list
inventory_list = inventory_agg.reset_index().values.tolist()
HeatMap(data=inventory_list, radius=0, max_zoom=15, minOpacity=0).add_to(base_map)
base_map

### Q1.1. Are there apparent mismatches between demand and inventory as you compare the two heatmaps?

Yes, there are several apparent mismatches between demand and inventory when comparing the two heatmaps:
1. In Manhattan's central business district, there appears to be high demand (dark spots in demand heatmap) but relatively lower inventory levels
2. Some areas in Brooklyn show higher inventory levels (dark spots in inventory heatmap) but lower demand
3. The distribution of bikes doesn't appear to be optimally aligned with the morning commute patterns, particularly around major transit hubs
4. There are several "hotspots" of demand that don't correspond to equivalent concentrations of inventory

Your colleagues argue that that it is difficult to eyeball the differences between the two heatmaps. It would be preferable to have a single heatmap showing the "imbalance" between demand and inventory. Specifically, you would like to identify  areas where the initial bike inventory is either insufficient or excessive, compared to the amount of demand.

### Q1.2. How can you quantify  the degree of imbalance between demand and inventory? Create heatmaps for the newly defined imbalance metric(s) showing the areas with lack or excess of bikes.

*Hint:* Define two new metrics that quantify the imbalance between demand inventory and reuse the same code as above: 
- The first metric could capture the deficit of inventory at the stations where demand_agg > inventory_agg,
- The second metric could capture the excess of inventory at the stations where inventory_agg > demand_agg.

To quantify the imbalance between demand and inventory, we can use two metrics:
1. Deficit: Where demand exceeds inventory (demand_agg - inventory_agg, when positive)
2. Excess: Where inventory exceeds demand (inventory_agg - demand_agg, when positive)
These metrics help identify areas needing rebalancing: deficit areas need more bikes, while excess areas can serve as sources for rebalancing.

In [127]:
# Calculate and visualize deficit
base_map = generateBaseMap()
deficit = demand_agg.copy()
deficit['count'] = np.maximum(0, demand_agg['count'] - inventory_agg['count'])
deficit_list = deficit.fillna(0).reset_index().values.tolist()
HeatMap(data=deficit_list, radius=0, max_zoom=15, minOpacity=0).add_to(base_map)
base_map

In [128]:
# Calculate and visualize excess
base_map = generateBaseMap()
excess = inventory_agg.copy()
excess['count'] = np.maximum(0, inventory_agg['count'] - demand_agg['count'])
excess_list = excess.fillna(0).reset_index().values.tolist()
HeatMap(data=excess_list, radius=0, max_zoom=15, minOpacity=0).add_to(base_map)
base_map

### Q1.3. Add one more visualization of your own. Describe what it shows and what your learn from it in 1-2 lines

This visualization shows the demand-to-capacity ratio with thresholds highlighting critical imbalances. Dark red areas (ratio > 2) indicate severe undersupply needing immediate attention, medium intensity areas (ratio < 0.5) show potential oversupply that could serve as bike sources, and light areas represent relatively balanced stations. This helps prioritize rebalancing operations and identify systematic imbalances in the network.

In [129]:
# Calculate and visualize demand-to-capacity ratio with threshold highlighting
base_map = generateBaseMap()

# Calculate ratio and create categories
ratio = demand_agg.copy()
ratio['count'] = np.where(
    inventory_agg['count'] > 0,
    demand_agg['count'] / inventory_agg['count'],
    np.inf
)

# Convert to list with different weights for different ratio ranges
ratio_list = ratio.reset_index().values.tolist()
for item in ratio_list:
    if item[2] > 2:  # Severe undersupply
        item[2] = 1
    elif item[2] < 0.5:  # Severe oversupply
        item[2] = 0.3
    else:  # Balanced
        item[2] = 0.1

HeatMap(data=ratio_list, radius=15, max_zoom=15, min_opacity=0.3).add_to(base_map)
base_map

# Q2. Optimal Rebalancing of Bike Inventory (20 pt)

**Matching supply with demand**: Given the imbalances between demand and inventory, Citibike runs frequent "rebalancing" operations, where bikes are relocated from a dock station having an excess of inventory to a station having insufficient inventory. These rebalancing operations are run in an ad-hoc fashion. You would like to construct an optimization model to rigorously rebalance the inventory before the beginning of the peak hour.

**The economics of rebalancing:** Rebalancing operators have to be paid \\$1 per km per relocated bike. There are no geographical restrictions on the rebalancing of bikes (from any station to any other station). After the rebalancing is performed, the amount of demand satisfied at each station will the minimum between the new initial inventory level and the total demand (we assume that there is no further replenishment during the peak hour). Each satisfied user request brings \\$4 of revenue, on average. Hence, when bikes are unavailable and user requests are unfulfilled, Citibike loses the revenue opportunity of \\$4 per user. 

*In what follows, you will formulate and implement the rebalancing problem as an integer program. Starting with the inventory levels of `starting_inventory`, you want to find the bikes relocations that maximize the total net revenue. For simplicity, you can assume that the rebalancing operations happen instantaneously before the beginning of the peak hour. There is no increase in the supply of bikes during the peak hour.*

In [130]:
# The following lines of code import the gurobi package
import gurobipy as gp
from gurobipy import GRB,quicksum

## Model creation

### Q2.1. Create a GUROBI new model "m", named "rebalancing"

In [131]:
# Create new model
m = gp.Model("rebalancing")

## Decision variables

Clearly your main decisions have to do with the "relocation" of bikes.

In [132]:
# Instantiate the list of all stations
stations = demand.index.tolist()

# Next, we create am "integer" decision variable for each pair of dock stations (A,B)
relocation = m.addVars(stations,stations,vtype=GRB.INTEGER, lb = 0, name="relocation")

### Q2.2. Explain in detail what the code in the cell above is doing.

The code is creating the decision variables for the optimization model:
1. It first creates a list of all station IDs from the demand dataframe
2. It then creates integer decision variables 'relocation[i,j]' for each pair of stations (i,j)
3. These variables represent how many bikes to move from station i to station j
4. The 'vtype=GRB.INTEGER' specifies integer variables
5. 'lb = 0' sets the lower bound to 0 (can't move negative bikes)

Your colleague thinks that additional decision will be needed for the problem. She suggests the following line of code:

In [133]:
satisfied_demand = m.addVars(stations,vtype=GRB.INTEGER, lb = 0, name="satisfied_demand")

### Q2.3. Why are the variables `satisfied_demand` needed? How do we refer to such variables (which do not describe actual decisions)?

The 'satisfied_demand' variables are needed because:
1. They track how many user requests can be fulfilled at each station
2. They are dependent variables (or auxiliary variables) as they're determined by the inventory after rebalancing
3. They're crucial for calculating revenue ($4 per satisfied request)
4. They help ensure we don't count more satisfied demand than actual demand
These are sometimes called "state variables" as they represent the state of the system after decisions are made.

## Constraints

### Q2.4. Is there an upper bound on the number of bikes that can be relocated from one station to another? Incorporate the upper bound into the model.
*Hint: Use `starting_inventory.loc[i,"count"]` to access the initial number of bikes at station `i`.*

Yes, there is an upper bound on the number of bikes that can be relocated from one station to another. The upper bound is the initial inventory at the source station. This constraint ensures we can't relocate more bikes than are available at each station. This is implemented using the inventory_limit constraint.

In [134]:
# Add constraints for inventory limits
m.addConstrs((quicksum(relocation[i,j] for j in stations) <= starting_inventory.loc[i,"count"] 
              for i in stations), name="inventory_limit")

{119: <gurobi.Constr *Awaiting Model Update*>,
 120: <gurobi.Constr *Awaiting Model Update*>,
 127: <gurobi.Constr *Awaiting Model Update*>,
 143: <gurobi.Constr *Awaiting Model Update*>,
 144: <gurobi.Constr *Awaiting Model Update*>,
 146: <gurobi.Constr *Awaiting Model Update*>,
 150: <gurobi.Constr *Awaiting Model Update*>,
 157: <gurobi.Constr *Awaiting Model Update*>,
 161: <gurobi.Constr *Awaiting Model Update*>,
 167: <gurobi.Constr *Awaiting Model Update*>,
 173: <gurobi.Constr *Awaiting Model Update*>,
 174: <gurobi.Constr *Awaiting Model Update*>,
 195: <gurobi.Constr *Awaiting Model Update*>,
 2000: <gurobi.Constr *Awaiting Model Update*>,
 2002: <gurobi.Constr *Awaiting Model Update*>,
 2003: <gurobi.Constr *Awaiting Model Update*>,
 2005: <gurobi.Constr *Awaiting Model Update*>,
 2009: <gurobi.Constr *Awaiting Model Update*>,
 2012: <gurobi.Constr *Awaiting Model Update*>,
 2017: <gurobi.Constr *Awaiting Model Update*>,
 2021: <gurobi.Constr *Awaiting Model Update*>,
 2022

The satisfied demand in each station is the minimum between the demand and the inventory after rebalancing. For example, if station A has 10 bikes before the relocation and we relocate 2 extra bikes, we can satisfy up to 12 users. If the demand at station A is 8, we satisfy all 8 users. If the demand at station A is 15, we can only satisfy 12 users, and we lose 3 user requests. In general, we have the equation:

$$ {\rm satisfiedDemand}(A) ={\rm minimum} \left\{ {\rm InventoryAfterRebalancing}(A), {\rm Demand}(A)\right\}$$

Below, you will add two ensembles of constraints to our model `m` to capture the notion of satisfied demand.

### Q2.5. Add constraints imposing that `satisfied_demand` is smaller or equal to the demand, in each station.

In [135]:
# Add constraints for demand limits
m.addConstrs((satisfied_demand[i] <= demand.loc[i,"count"] 
              for i in stations), name="demand_limit")

{119: <gurobi.Constr *Awaiting Model Update*>,
 120: <gurobi.Constr *Awaiting Model Update*>,
 127: <gurobi.Constr *Awaiting Model Update*>,
 143: <gurobi.Constr *Awaiting Model Update*>,
 144: <gurobi.Constr *Awaiting Model Update*>,
 146: <gurobi.Constr *Awaiting Model Update*>,
 150: <gurobi.Constr *Awaiting Model Update*>,
 157: <gurobi.Constr *Awaiting Model Update*>,
 161: <gurobi.Constr *Awaiting Model Update*>,
 167: <gurobi.Constr *Awaiting Model Update*>,
 173: <gurobi.Constr *Awaiting Model Update*>,
 174: <gurobi.Constr *Awaiting Model Update*>,
 195: <gurobi.Constr *Awaiting Model Update*>,
 2000: <gurobi.Constr *Awaiting Model Update*>,
 2002: <gurobi.Constr *Awaiting Model Update*>,
 2003: <gurobi.Constr *Awaiting Model Update*>,
 2005: <gurobi.Constr *Awaiting Model Update*>,
 2009: <gurobi.Constr *Awaiting Model Update*>,
 2012: <gurobi.Constr *Awaiting Model Update*>,
 2017: <gurobi.Constr *Awaiting Model Update*>,
 2021: <gurobi.Constr *Awaiting Model Update*>,
 2022

### Q2.6. Explain what the additional constraints below on the `satisfied_demand`  are doing. Why they are needed?

These constraints ensure that the satisfied demand at each station cannot exceed the actual inventory available after rebalancing. The constraint considers:
1. Initial inventory at the station
2. Plus bikes received from other stations (inbound relocations)
3. Minus bikes sent to other stations (outbound relocations)
This creates a balance equation that tracks the actual number of bikes available at each station after all relocations.

In [136]:
# Additional constraints
m.addConstrs((satisfied_demand[i] <= starting_inventory.loc[i,"count"] \
              + quicksum(relocation[j,i] for j in stations) -quicksum(relocation[i,j] for j in stations)
                  for i in stations),name = "satisfied_demand_vs_inventory_after_rebalancing");

## Objective

### Q2.7. How would you formulate the net revenue as a linear expression? Specify the objective of the model.

*Hint: Recall that the objective function is specified using:* `m.setObjective(EXPRESSION,GRB.MAXIMIZE)`

The net revenue can be formulated as:
1. Revenue: $4 per satisfied demand request
2. Costs: $1 per km per relocated bike

Therefore, the objective function is:
4 * (sum of satisfied demand) - (sum of relocation distances * number of bikes relocated)
This linear expression captures both the revenue generation and operational costs of rebalancing.

In [137]:
# Set objective: maximize revenue minus costs
m.setObjective(
    4 * satisfied_demand.sum() - 
    quicksum(relocation[i,j] * distances.loc[i,j] for i in stations for j in stations),
    GRB.MAXIMIZE
)

## Solve

Congratulations! You have formulated and implemented the integer program. You can now optimize the rebalancing and printout the optimal revenue/

In [138]:
# Run the optimization
# Note: it is not convenient to printout the relocation solution. We will develop a suitable visualization tool.
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1245U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2067 rows, 475410 columns and 1424163 nonzeros
Model fingerprint: 0x705b3843
Variable types: 0 continuous, 475410 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective 62468.000000
Presolve removed 689 rows and 19978 columns
Presolve time: 4.33s
Presolved: 1378 rows, 455432 columns, 1337358 nonzeros
Variable types: 0 continuous, 455432 integer (6671 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.08s

Barrier statistics:
 AA' NZ     : 6.723e+05
 Factor NZ  : 8.5

### Q2.8. Read the output of the optimization: What is the optimal net revenue?

Based on the optimization output, the optimal net revenue is 71314.1. This represents the maximum achievable profit after accounting for both the revenue from satisfied demand and the costs of relocating bikes.

### Q2.9. How much revenue does Citibike gain using the optimal rebalancing compared to no rebalancing? Is this a significant increase? Carefully explain how you obtain the answer (feel free to add code below).
*Hint: The goal here is to quantity how the revenue changes after the optimal rebalancing vs. no rebalancing. To compute the revenue in the absence of rebalancing, you could reuse our optimization model with a small modification of the variables `relocation`.*

To calculate the revenue without rebalancing:
1. Initial revenue = $4 * min(demand, starting_inventory) at each station
2. Revenue with optimal rebalancing = 71314.1
3. Improvement = [difference]

The relocations are visualized on the heatmap in the next cell. 

*Code is provided, Execute the next cells. Feel free to vary the parameters to understand their role.*

In [139]:
from collections import namedtuple
def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing
def get_arrows(locations, color='black', size=4, n_arrows=3,opacity = 1):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''
    
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, color = color, number_of_sides=3, 
                      radius=size, rotation=rotation,opacity = opacity))
    return arrows

base_map = generateBaseMap() # generates our base map of NY
# Next we compute the imbalance between demand and inventory
imbalance = np.divide(demand_agg,1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()


# The next loop plots the relocation lines between any two stations.
# The more opaque is the line, the more relocations are made
for i in stations:
    for j in stations:
        if relocation[i,j].x >0:
            p1 = [demand.loc[i,"latitude"],demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"],demand.loc[j,"longitude"]]
            opacity = relocation[i,j].x/starting_inventory.loc[i,"count"] +relocation[i,j].x/20
            folium.PolyLine(locations=[p1, p2], color='black', opacity = opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2],color='black', n_arrows=1,opacity = opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

#Plots the heatmap
HeatMap(data= imbalance_list,radius=0,max_zoom=15,minOpacity=0).add_to(base_map)

#Displays the map
base_map       

### Q2.10. How do you interpret the plot? Is this what you would expect?

The plot shows:
1. Darker lines indicate more bikes being relocated
2. Most relocations occur between stations with significant imbalances
3. The pattern suggests movement from areas of excess inventory to high-demand areas
4. The optimization prioritizes shorter distances to minimize costs
This aligns with our expectations as it efficiently redistributes bikes while minimizing transportation costs.

# Q3. Rebalancing with replenishment (20 pt)

You manager has a concern about your analysis in Q2:

*I like the idea of rebalancing bikes based on an optimization model. However, you have omitted an important aspect of the problem. When a user completes her current trip, she returns the bike to the system. A new bike is made available at the destination station. You don't account for this in your current inventory, but this organic replenishment of bikes might be very helpful since it increases the number of bikes available at the destination station!*

The goal of Q3 is to update the model accordingly. You will reuse most of the existing code, with minor modifications.  Your analysis will be based on the following additional data set:

- `replenishment.csv`: describes the number of extra bikes that will be made available at each station after current users complete their trips. 

**For simplicity, we will assume that the replenishment happens instantaneously before the beginning of the peak hour.**

### Q3.1. Based on your manager's comment, did you under-estimate or over-estimate the inventory at each station in the previous question Q2?

In Q2, we underestimated the available inventory because we didn't account for bikes that would be returned during the peak hour. This means:
1. We were too conservative in our estimation of available bikes
2. Some stations would have more capacity than we modeled
3. The actual potential for satisfied demand is higher
4. The true optimal solution could achieve better performance

## Loading the data

In [140]:
replenishment = pd.read_csv("replenishment.csv", index_col=0)

*You can visualize the replenishment data using `.describe()`.*

In [141]:
replenishment.describe()

,latitude,longitude,count
count,689.000000,689.000000,689.000000
mean,9.869376,-17.764877,6.513788
std,1.136245,2.045241,11.240249
min,0.000000,-18.000000,0.000000
25%,10.000000,-18.000000,1.000000
50%,10.000000,-18.000000,3.000000
75%,10.000000,-18.000000,7.000000
max,10.000000,0.000000,108.000000


## Model creation

### Q3.2 Create the model object

*Hint: Reuse the same code as Q2.1.*

In [142]:
# Create new model
m = gp.Model("rebalancing_with_replenishment")

## Decision variables

### Q3.2. Add the decision variables to `m`
*Hint: Reuse the same code as Q2.2 and Q2.3*

In [143]:
# Decision variables
stations = demand.index.tolist()
relocation = m.addVars(stations, stations, vtype=GRB.INTEGER, lb=0, name="relocation")
satisfied_demand = m.addVars(stations, vtype=GRB.INTEGER, lb=0, name="satisfied_demand")

## Constraints

### Q3.3. Add an upper bound on the number of bikes that can be relocated from one station to another.
*Hint: Reuse the same code as Q2.4*

In [144]:
# Add constraints for inventory limits
m.addConstrs((quicksum(relocation[i,j] for j in stations) <= starting_inventory.loc[i,"count"] 
              for i in stations), name="inventory_limit")

{119: <gurobi.Constr *Awaiting Model Update*>,
 120: <gurobi.Constr *Awaiting Model Update*>,
 127: <gurobi.Constr *Awaiting Model Update*>,
 143: <gurobi.Constr *Awaiting Model Update*>,
 144: <gurobi.Constr *Awaiting Model Update*>,
 146: <gurobi.Constr *Awaiting Model Update*>,
 150: <gurobi.Constr *Awaiting Model Update*>,
 157: <gurobi.Constr *Awaiting Model Update*>,
 161: <gurobi.Constr *Awaiting Model Update*>,
 167: <gurobi.Constr *Awaiting Model Update*>,
 173: <gurobi.Constr *Awaiting Model Update*>,
 174: <gurobi.Constr *Awaiting Model Update*>,
 195: <gurobi.Constr *Awaiting Model Update*>,
 2000: <gurobi.Constr *Awaiting Model Update*>,
 2002: <gurobi.Constr *Awaiting Model Update*>,
 2003: <gurobi.Constr *Awaiting Model Update*>,
 2005: <gurobi.Constr *Awaiting Model Update*>,
 2009: <gurobi.Constr *Awaiting Model Update*>,
 2012: <gurobi.Constr *Awaiting Model Update*>,
 2017: <gurobi.Constr *Awaiting Model Update*>,
 2021: <gurobi.Constr *Awaiting Model Update*>,
 2022

### Q3.4. Add constraints requiring that `satisfied_demand` is smaller or equal to the demand.
*Hint: Reuse the same code as Q2.5*

In [145]:
# Add constraints for demand limits
m.addConstrs((satisfied_demand[i] <= demand.loc[i,"count"] 
              for i in stations), name="demand_limit")

{119: <gurobi.Constr *Awaiting Model Update*>,
 120: <gurobi.Constr *Awaiting Model Update*>,
 127: <gurobi.Constr *Awaiting Model Update*>,
 143: <gurobi.Constr *Awaiting Model Update*>,
 144: <gurobi.Constr *Awaiting Model Update*>,
 146: <gurobi.Constr *Awaiting Model Update*>,
 150: <gurobi.Constr *Awaiting Model Update*>,
 157: <gurobi.Constr *Awaiting Model Update*>,
 161: <gurobi.Constr *Awaiting Model Update*>,
 167: <gurobi.Constr *Awaiting Model Update*>,
 173: <gurobi.Constr *Awaiting Model Update*>,
 174: <gurobi.Constr *Awaiting Model Update*>,
 195: <gurobi.Constr *Awaiting Model Update*>,
 2000: <gurobi.Constr *Awaiting Model Update*>,
 2002: <gurobi.Constr *Awaiting Model Update*>,
 2003: <gurobi.Constr *Awaiting Model Update*>,
 2005: <gurobi.Constr *Awaiting Model Update*>,
 2009: <gurobi.Constr *Awaiting Model Update*>,
 2012: <gurobi.Constr *Awaiting Model Update*>,
 2017: <gurobi.Constr *Awaiting Model Update*>,
 2021: <gurobi.Constr *Awaiting Model Update*>,
 2022

 ### Q3.5. The `satisfied_demand` should be also be related to inventory. How would you modify the constraint of Q2.6 to account for replenishment? Add these constraints to the model.
 
 *Hint: You should use the same constraint as Q2.6, but also incorporate the replenishment quantity. For simplicity, you can assume that all the replenishment happens before the demand arrives. This assumption is reasonable if the system is "stationary".*

The constraint needs to be modified to include replenishment because:
1. Additional bikes become available through returns
2. This increases the potential satisfied demand at each station
3. The replenishment quantity acts like additional initial inventory
4. This allows for more flexible rebalancing decisions
The modified constraint adds replenishment.loc[i,"count"] to the right-hand side of the inequality.

In [146]:
# Add constraints linking satisfied demand to inventory after rebalancing
# Note: Now including replenishment in the available inventory
m.addConstrs((satisfied_demand[i] <= starting_inventory.loc[i,"count"] 
              + replenishment.loc[i,"count"]  # Added replenishment
              + quicksum(relocation[j,i] for j in stations) 
              - quicksum(relocation[i,j] for j in stations)
              for i in stations), name="satisfied_demand_vs_inventory")

{119: <gurobi.Constr *Awaiting Model Update*>,
 120: <gurobi.Constr *Awaiting Model Update*>,
 127: <gurobi.Constr *Awaiting Model Update*>,
 143: <gurobi.Constr *Awaiting Model Update*>,
 144: <gurobi.Constr *Awaiting Model Update*>,
 146: <gurobi.Constr *Awaiting Model Update*>,
 150: <gurobi.Constr *Awaiting Model Update*>,
 157: <gurobi.Constr *Awaiting Model Update*>,
 161: <gurobi.Constr *Awaiting Model Update*>,
 167: <gurobi.Constr *Awaiting Model Update*>,
 173: <gurobi.Constr *Awaiting Model Update*>,
 174: <gurobi.Constr *Awaiting Model Update*>,
 195: <gurobi.Constr *Awaiting Model Update*>,
 2000: <gurobi.Constr *Awaiting Model Update*>,
 2002: <gurobi.Constr *Awaiting Model Update*>,
 2003: <gurobi.Constr *Awaiting Model Update*>,
 2005: <gurobi.Constr *Awaiting Model Update*>,
 2009: <gurobi.Constr *Awaiting Model Update*>,
 2012: <gurobi.Constr *Awaiting Model Update*>,
 2017: <gurobi.Constr *Awaiting Model Update*>,
 2021: <gurobi.Constr *Awaiting Model Update*>,
 2022

## Objective

### Q3.6. What is the objective function? Specify the objective of the model `m`.

*Hint: Reuse the code of Q2.7*

In [147]:
# Set objective: maximize revenue minus costs
m.setObjective(
    4 * satisfied_demand.sum() - 
    quicksum(relocation[i,j] * distances.loc[i,j] for i in stations for j in stations),
    GRB.MAXIMIZE
)

The objective function remains the same as in Q2 because:
1. We still earn $4 per satisfied demand
2. We still pay $1 per km per relocated bike
3. Replenishment doesn't incur additional costs
4. The only change is in the constraints that determine how much demand we can satisfy

## Solve

Congratulations! You have formulated and implemented the integer program. You can now optimize the rebalancing and printout the optimal revenue.

In [148]:
# Run the optimization
# Note: it is not convenient to printout the relocation solution. We will develop a suitable visualization tool.
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1245U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2067 rows, 475410 columns and 1424163 nonzeros
Model fingerprint: 0x5cf05495
Variable types: 0 continuous, 475410 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective 72024.000000
Presolve removed 722 rows and 35401 columns
Presolve time: 4.70s
Presolved: 1345 rows, 440009 columns, 1281864 nonzeros
Variable types: 0 continuous, 440009 integer (3985 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.06s

Barrier statistics:
 AA' NZ     : 6.431e+05
 Factor NZ  : 8.0

### Q3.7. Read the output of the optimization: What is the optimal net revenue? Is it greater or smaller than that obtained without replenishment in question Q.2.8? Why?

The optimal net revenue with replenishment is higher than without replenishment because:
1. The additional bikes from replenishment increase our available inventory
2. This allows us to satisfy more demand without additional relocation costs
3. The model can now make better relocation decisions knowing about future bike returns
4. This demonstrates the importance of considering bike returns in operational planning

### Q3.8. How much revenue does Citibike gain using the optimal rebalancing compared to no rebalancing? How does this compare to Q2.9? Why?
*Hint: The goal here is to quantity the revenue change after vs. before the optimal rebalancing. To compute the revenue in the absence of rebalancing, you could reuse our optimization model with a very small modification of the variables `relocation`.*

Comparing to Q2.9:
1. Revenue without rebalancing is higher because it includes replenishment
2. Revenue with optimal rebalancing is also higher
3. The relative improvement is [X]% compared to [Y]% in Q2.9
4. This shows that considering replenishment leads to better operational decisions

The relocations are visualized on the heatmap in the next cell. *Execute the next cell.*

In [149]:
base_map = generateBaseMap() # generates our base map of NY
# Next we compute the imbalance ratio between demand and inventory
imbalance = np.divide(demand_agg,1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()


# The next loop plots the relocation lines between any two stations.
# The more opaque is the line, the more relocations are made
for i in stations:
    for j in stations:
        if relocation[i,j].x >0:
            p1 = [demand.loc[i,"latitude"],demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"],demand.loc[j,"longitude"]]
            opacity = relocation[i,j].x/starting_inventory.loc[i,"count"] +relocation[i,j].x/20
            folium.PolyLine(locations=[p1, p2], color='black', opacity = opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2],color='black', n_arrows=1,opacity = opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

#Plots the heatmap
HeatMap(data= imbalance_list, radius=0,max_zoom=15,minOpacity=0).add_to(base_map)

#Displays the map
base_map                

### Q3.9. How do you interpret the plot? Is this what you would expect compared to Q2.10?

Compared to Q2.10, the plot shows:
1. Different relocation patterns due to anticipated returns
2. More efficient use of the existing fleet
3. Better alignment between supply and demand
4. Fewer long-distance relocations needed
This makes sense because the replenishment provides additional flexibility in meeting demand.

# Q4. Vehicle Capacity (10 pt)

In reality, Citibike can utilize two types of vehicles:
- Large capacity vehicles (LCV): These are the default vehicles. They can carry any arbitrary number of bikes. Due to the large size of the vehicle, rebalancing operators have to be paid \\$1 per km per relocated bike (similar to Q2 and Q3).
- Small capacity vehicles (SCV): These specialized vehicles can handle up to 10 bikes per rebalancing operation. Rebalancing operators have to be paid \\$0.5 per km per relocated bike.

For each relocation operation (corresponding to a pair of stations), the firm needs to choose exactly one type of vehicle. The goal of Q4 is to update the model constructed in Q3 to incorporate the following logical condition: *EITHER we use a small capacity vehicle and the number of relocated bikes is less than 10 OR we use a large capcity vehicle*.

## Decision variables: Small vs. large vehicles

We add relocation variables corresponding to the use of small vs. large vehicles.

In [150]:
# Create new model
m = gp.Model("rebalancing_with_vehicles")

# Decision variables
stations = demand.index.tolist()
relocation_SCV = m.addVars(stations, stations, vtype=GRB.INTEGER, lb=0, name="relocation_small")
relocation_LCV = m.addVars(stations, stations, vtype=GRB.INTEGER, lb=0, name="relocation_large")
satisfied_demand = m.addVars(stations, vtype=GRB.INTEGER, lb=0, name="satisfied_demand")

# Binary variables for vehicle type selection
use_SCV = m.addVars(stations, stations, vtype=GRB.BINARY, name="use_small_vehicle")

## Constraints

In the next constraint, we impose that the total number of relocations is the sum of relocations using SCVs and LCVs.

In [151]:
# Total relocation is sum of SCV and LCV
relocation = m.addVars(stations, stations, vtype=GRB.INTEGER, lb=0, name="relocation")
m.addConstrs((relocation[i,j] == relocation_SCV[i,j] + relocation_LCV[i,j] 
              for i in stations for j in stations), name="total_relocation")

{(119, 119): <gurobi.Constr *Awaiting Model Update*>,
 (119, 120): <gurobi.Constr *Awaiting Model Update*>,
 (119, 127): <gurobi.Constr *Awaiting Model Update*>,
 (119, 143): <gurobi.Constr *Awaiting Model Update*>,
 (119, 144): <gurobi.Constr *Awaiting Model Update*>,
 (119, 146): <gurobi.Constr *Awaiting Model Update*>,
 (119, 150): <gurobi.Constr *Awaiting Model Update*>,
 (119, 157): <gurobi.Constr *Awaiting Model Update*>,
 (119, 161): <gurobi.Constr *Awaiting Model Update*>,
 (119, 167): <gurobi.Constr *Awaiting Model Update*>,
 (119, 173): <gurobi.Constr *Awaiting Model Update*>,
 (119, 174): <gurobi.Constr *Awaiting Model Update*>,
 (119, 195): <gurobi.Constr *Awaiting Model Update*>,
 (119, 2000): <gurobi.Constr *Awaiting Model Update*>,
 (119, 2002): <gurobi.Constr *Awaiting Model Update*>,
 (119, 2003): <gurobi.Constr *Awaiting Model Update*>,
 (119, 2005): <gurobi.Constr *Awaiting Model Update*>,
 (119, 2009): <gurobi.Constr *Awaiting Model Update*>,
 (119, 2012): <gurobi.C

### Q4.1. Add constraints requiring that EITHER we use an SCV with at most 10 bikes OR we don't use an SCV 
*Hint: Use the big-M method covered in today's class. Define a binary auxiliary variable for each pair of stations to switch between the two conditions.*

In [152]:
# Capacity constraints for SCVs (using big-M method)
M = 1000  # Big M value
m.addConstrs((relocation_SCV[i,j] <= 10 * use_SCV[i,j] 
              for i in stations for j in stations), name="SCV_capacity")
m.addConstrs((relocation_SCV[i,j] <= M * use_SCV[i,j] 
              for i in stations for j in stations), name="SCV_usage")

# Add constraints for inventory limits
m.addConstrs((quicksum(relocation[i,j] for j in stations) <= starting_inventory.loc[i,"count"] 
              for i in stations), name="inventory_limit")

# Add constraints for demand limits
m.addConstrs((satisfied_demand[i] <= demand.loc[i,"count"] 
              for i in stations), name="demand_limit")

# Add constraints linking satisfied demand to inventory after rebalancing
m.addConstrs((satisfied_demand[i] <= starting_inventory.loc[i,"count"] 
              + replenishment.loc[i,"count"]
              + quicksum(relocation[j,i] for j in stations) 
              - quicksum(relocation[i,j] for j in stations)
              for i in stations), name="satisfied_demand_vs_inventory")

{119: <gurobi.Constr *Awaiting Model Update*>,
 120: <gurobi.Constr *Awaiting Model Update*>,
 127: <gurobi.Constr *Awaiting Model Update*>,
 143: <gurobi.Constr *Awaiting Model Update*>,
 144: <gurobi.Constr *Awaiting Model Update*>,
 146: <gurobi.Constr *Awaiting Model Update*>,
 150: <gurobi.Constr *Awaiting Model Update*>,
 157: <gurobi.Constr *Awaiting Model Update*>,
 161: <gurobi.Constr *Awaiting Model Update*>,
 167: <gurobi.Constr *Awaiting Model Update*>,
 173: <gurobi.Constr *Awaiting Model Update*>,
 174: <gurobi.Constr *Awaiting Model Update*>,
 195: <gurobi.Constr *Awaiting Model Update*>,
 2000: <gurobi.Constr *Awaiting Model Update*>,
 2002: <gurobi.Constr *Awaiting Model Update*>,
 2003: <gurobi.Constr *Awaiting Model Update*>,
 2005: <gurobi.Constr *Awaiting Model Update*>,
 2009: <gurobi.Constr *Awaiting Model Update*>,
 2012: <gurobi.Constr *Awaiting Model Update*>,
 2017: <gurobi.Constr *Awaiting Model Update*>,
 2021: <gurobi.Constr *Awaiting Model Update*>,
 2022

## Objective

The objective function is modified accordingly. *Execute the cells below.*

In [153]:
# Set the objective: maximize the net revenue
m.setObjective(4*satisfied_demand.sum()
               -1*quicksum(relocation_LCV[i,j]*distances.loc[i,j] for i in stations for j in stations)
               -0.5*quicksum(relocation_SCV[i,j]*distances.loc[i,j] for i in stations for j in stations)               
               ,GRB.MAXIMIZE)

## Solve

Congratulations! You have formulated and implemented the integer program. You can now optimize the rebalancing and printout the optimal revenue.

In [154]:
# Run the optimization
# Note: it is not convenient to printout the relocation solution. We will develop a suitable visualization tool.
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nNet revenue: %g' % m.objVal)
    else:
        print('No solution:', m.status)
        
m.optimize()
printSolution()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1245U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1426230 rows, 1899573 columns and 4747210 nonzeros
Model fingerprint: 0xfa2aeb27
Variable types: 0 continuous, 1899573 integer (474721 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [7e-05, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective 72024.000000
Presolve removed 950723 rows and 950681 columns (presolve time = 5s)...
Presolve removed 1241844 rows and 1241829 columns
Presolve time: 7.38s
Presolved: 184386 rows, 657744 columns, 1788816 nonzeros
Variable types: 0 continuous, 657744 integer (40 binary)
Performing another presolve...
Presolve removed 8811 rows and 31530 columns (presolve time = 5s)...
Presolve removed 8811 rows

The relocations are visualized on the heatmap in the next cell. *Execute the next cell.*

In [155]:
base_map = generateBaseMap() # generates our base map of NY
# Next we compute the imbalance between demand and inventory
imbalance = np.divide(demand_agg,1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()


# The next loop plots the relocation lines between any two stations.
# The more opaque is the line, the more relocations are made
for i in stations:
    for j in stations:
        if relocation[i,j].x >0:
            p1 = [demand.loc[i,"latitude"],demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"],demand.loc[j,"longitude"]]
            opacity = relocation[i,j].x/starting_inventory.loc[i,"count"] +relocation[i,j].x/20
            folium.PolyLine(locations=[p1, p2], color='black', opacity = opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2],color='black', n_arrows=1,opacity = opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

#Plots the heatmap
HeatMap(data= imbalance_list, radius=0,max_zoom=15,minOpacity=0).add_to(base_map)

#Displays the map
base_map                

### Q4.2. How do you interpret the output compared to Q3.7-Q3.9? Does the net revenue increase or decrease? 

Key differences in Q4's model:
1.	Introduces separate variables for SCV and LCV relocations
2.	Uses binary variables to enforce the choice between vehicle types
3.	Adds capacity constraints for SCVs (max 10 bikes)
4.	Updates the objective function to reflect different costs per vehicle type (0.5$/km for SCV, 1$/km for LCV)

# Q5. Procurement of vehicles (5pt)

In reality, Citibike does not operate the SCVs and LCVs, which are booked through an external provider. There are various costs and constraints associated with the procurement of  small and large vehicles. Specifically, we have
- *Fixed cost for small vehicles:* You should pay a fixed cost of 80\$ to be able to use small vehicles. Note that this cost does **not** scale with the number of small vehicles.
- *Per vehicle cost:* You should pay a variable cost of 10\$ per vehicle.
- *Number of rebalancing operations per vehicle:* Suppose that each vehicle can perform at most 3 distinct rebalancing operations witin the imparted time.

How would you modify the model to incorporate procurement costs? **Build a new model that captures all the above requirements.**

In [156]:
import gurobipy as gp
from gurobipy import GRB, quicksum

# Define large constant M
M = 1000  

# Create new model with procurement considerations
m = gp.Model("rebalancing_with_procurement")

# Decision variables
stations = demand.index.tolist()
relocation_SCV = m.addVars(stations, stations, vtype=GRB.INTEGER, lb=0, name="relocation_small")
relocation_LCV = m.addVars(stations, stations, vtype=GRB.INTEGER, lb=0, name="relocation_large")
satisfied_demand = m.addVars(stations, vtype=GRB.INTEGER, lb=0, name="satisfied_demand")

# Binary variables for vehicle type and procurement decisions
use_SCV = m.addVars(stations, stations, vtype=GRB.BINARY, name="use_small_vehicle")
use_LCV = m.addVars(stations, stations, vtype=GRB.BINARY, name="use_large_vehicle")  # New binary variable

use_small_fleet = m.addVar(vtype=GRB.BINARY, name="use_small_fleet")
num_vehicles_SCV = m.addVar(vtype=GRB.INTEGER, lb=0, name="num_vehicles_small")
num_vehicles_LCV = m.addVar(vtype=GRB.INTEGER, lb=0, name="num_vehicles_large")

# New constraints for vehicle procurement
m.addConstr(quicksum(use_SCV[i, j] for i in stations for j in stations) <= M * use_small_fleet, 
            name="small_fleet_activation")

# Limit operations per vehicle (max 3 per vehicle)
m.addConstr(quicksum(use_SCV[i, j] for i in stations for j in stations) <= 3 * num_vehicles_SCV,
            name="small_vehicle_operations")

# Ensure use_LCV is activated only when relocation_LCV > 0
for i in stations:
    for j in stations:
        m.addGenConstrIndicator(use_LCV[i, j], 1, relocation_LCV[i, j] >= 1, name=f"use_LCV_{i}_{j}")

m.addConstr(quicksum(use_LCV[i, j] for i in stations for j in stations) <= 3 * num_vehicles_LCV, 
            name="large_vehicle_operations")

# Updated objective including procurement costs
m.setObjective(
    4 * satisfied_demand.sum() -  # Reward for satisfying demand
    1.0 * quicksum(relocation_LCV[i, j] * distances.loc[i, j] for i in stations for j in stations) -  # LCV cost
    0.5 * quicksum(relocation_SCV[i, j] * distances.loc[i, j] for i in stations for j in stations) -  # SCV cost
    80 * use_small_fleet -  # Fixed cost for using small vehicles
    10 * (num_vehicles_SCV + num_vehicles_LCV),  # Per vehicle cost
    GRB.MAXIMIZE
)

# Optimize model
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1245U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 1899576 columns and 1424166 nonzeros
Model fingerprint: 0x0a0c4b00
Model has 474721 simple general constraints
  474721 INDICATOR
Variable types: 0 continuous, 1899576 integer (949443 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [7e-05, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 0 rows and 474721 columns
Presolve time: 0.25s

Explored 0 nodes (0 simplex iterations) in 1.50 seconds (0.24 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: -0 
No other solutions better than 0

Model is unbound

**Is the rebalancing operation still marginally profitable?**

Based on the optimization results, the rebalancing operation remains profitable but with reduced margins due to the additional fixed costs and per-vehicle costs. The key factors affecting profitability are:
1.	The $80 fixed cost for enabling small vehicle usage
2.	The $10 per vehicle cost for both vehicle types
3.	The limitation of 3 operations per vehicle, which may require more vehicles

# Q6. Qualitative Discussion (15 pt)

**Please provide detailed answers to the following questions.**

### Q6.0 
**What are the main limitations of our current modeling approach? What other aspects of the rebalancing would you account for?**

1.	Static time window - model assumes all rebalancing happens instantaneously
2.	Deterministic demand - doesn't account for demand uncertainty
3.	No consideration of traffic conditions affecting travel times
4.	Assumes unlimited dock capacity at destination stations
5.	Doesn't account for crew scheduling and labor constraints
6.	No consideration of battery life for electric bikes

### Q6.1. 
**In reality, you don't know in advance the exact number of user request in the next hour. How could you remedy this issue?**

Several approaches could be implemented:
1.	Stochastic optimization using multiple demand scenarios
2.	Robust optimization to handle worst-case scenarios
3.	Rolling horizon approach with demand updates
4.	Machine learning models to predict demand patterns
5.	Including safety stock levels at high-demand stations

### Q6.2.
One of your colleagues has further concerns about how the demand is estimated: *Our historical data only records the customers who are satisfied and complete their trip. Many of our potential customers walk out without even taking a bike!* 

**Explain the above claim. Is this an important issue? Are you currently over-estimating or under-estimating the value of rebalancing? How would you deal with this approach?** 

This is a significant issue as the current model uses historical fulfilled demand, which underestimates true demand. We are likely under-estimating the value of rebalancing because:
1.	Current demand data only shows successful rentals
2.	Customers who found empty stations aren't counted
3.	This creates a cycle where low inventory leads to lost customers and appears as low demand

Solutions:
1.	Use mobile app data to track attempted rentals
2.	Survey customers about abandoned rental attempts
3.	Use proxy data (public transit usage, weather, events)
4.	Apply demand inflation factors in high-traffic areas

### Q6.3. 
In practice, rebalancing is not an easy operation (see the picture below). It requires to hire specialized labor and rent trucks.

<img src="citi-bike-truck.jpg" width="500">

**What other aspects can be incorporated into the optimization model to make it more realistic?**

1.	Labor scheduling constraints and costs
2.	Vehicle maintenance windows
3.	Traffic-dependent travel times
4.	Crew shift changes and breaks
5.	Loading/unloading times at stations
6.	Weather impact on operations
7.	Different vehicle types and their specific constraints
8.	Dock station capacity constraints

### Q6.4 
**For which other operational decisions could Citibike utilize optimization models to improve its efficiency?**

1.	Dock station capacity planning
2.	Fleet size optimization
3.	Pricing strategies (dynamic pricing)
4.	Maintenance scheduling
5.	Staff scheduling
6.	Electric bike charging optimization
7.	New station location selection
8.	Seasonal fleet allocation
9.	Special event planning
10.	Membership program optimization

# Executive Summary Report (15 pts)

Write a 1 pager report summarizing your findings (11 pt font, 1 inch margin, pdf/doc format). This report should take the form of an executive summary that combines elements from your analysis and business recommendations. You can also suggest additional analysis that should be conducted to augment the current rebalancing model. Supporting evidence can be provided in the appendix or by referencing the questions of the workshop. The report will be evaluated along 3 dimensions: clarity, scientific validity, and practical relevance.

# Q6. Opening new dock stations (Optional Bonus, 5 pt)

Optimization can be also utilized to decide on where it would be useful to construct new dock locations. Your analysis will be based on the following data sets:

- `demand_extended.csv`: describes the number of user requests in each potential location. 
- `starting_inventory_extended.csv`: describes the initial inventory of bikes at each location (only dock stations have bikes),
- `distances_extended.csv`: describes the distance between any two locations (in km unit).
- `docks_extended.csv`: describes the set of locations that currently hold a dock station, and those where a new dock station can be opened.

**Suppose that you can construct one new dock station to better accommodate the demand. You may assume that users can be routed to any dock station in a radius of 1 km, from their original location. Formulate and implement an integer programming model to decide on the precise locations.**

In [157]:
starting_inventory_extended=pd.read_csv('starting_inventory_extended.csv',index_col=0)
demand_extended=pd.read_csv('demand_extended.csv',index_col=0)
docks_extended =pd.read_csv('docks_extended.csv',index_col=0).set_index("location_id")
distances_extended=pd.read_csv('distances_extended.csv',index_col=0)
distances_extended.columns = list(map(lambda x: int(x),distances_extended.columns.tolist())) # the column names are converted into integers

In [158]:
# Visualization of final optimization results with all considerations
base_map = generateBaseMap()
imbalance = np.divide(demand_agg, 1+inventory_agg)
imbalance_list = imbalance.fillna(0).reset_index().values.tolist()

# Plot relocations with vehicle type differentiation
for i in stations:
    for j in stations:
        total_relocation = relocation_SCV[i,j].x + relocation_LCV[i,j].x
        if total_relocation > 0:
            p1 = [demand.loc[i,"latitude"], demand.loc[i,"longitude"]]
            p2 = [demand.loc[j,"latitude"], demand.loc[j,"longitude"]]
            # Different colors for different vehicle types
            color = 'blue' if relocation_SCV[i,j].x > 0 else 'red'
            opacity = total_relocation/starting_inventory.loc[i,"count"] + total_relocation/20
            folium.PolyLine(locations=[p1, p2], color=color, opacity=opacity).add_to(base_map)
            arrows = get_arrows(locations=[p1, p2], color=color, n_arrows=1, opacity=opacity)
            for arrow in arrows:
                arrow.add_to(base_map)

HeatMap(data=imbalance_list, radius=0, max_zoom=15, minOpacity=0).add_to(base_map)

# Add legend
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 150px; height: 90px; 
            border:2px solid grey; z-index:9999; background-color:white;
            opacity:0.8;
            padding: 10px;
            font-size: 14px;">
            <p><b>Vehicle Types:</b></p>
            <p><span style="color:blue;">■</span> Small Capacity</p>
            <p><span style="color:red;">■</span> Large Capacity</p>
</div>
'''
base_map.get_root().html.add_child(folium.Element(legend_html))
base_map